In [1]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig
import json
from Gpt4_OpenAI_Prompt import system_msg,context,postprocess

In [2]:
from Newprompt import system_msg as newprompt

In [3]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

True
Tesla T4


In [4]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)
# Load the entire model on the GPU 0
device_map = {"": 0}


In [5]:
model_name = "NousResearch/llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(model_name,quantization_config=bnb_config,
    device_map=device_map)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
#Testing prompt

# question = "what are the sales in africa"
# prompt = f"""[INST]<<SYS>>
# Assume you are a system that detects the month and year in the given secentance, user will give only provide the input in english.
# For example if user gives "top  orders in march 2020", you should only return march 2020.

# <</SYS>>
# {question}
# [/INST]
# [END]"""

In [7]:
def gereneate_output_llama2(prompt):
    #prompt = json.dumps(prompt)
    model_input = tokenizer(prompt, return_tensors="pt").to("cuda")
    #model.eval()
    response = tokenizer.decode(model.generate(**model_input, max_length=4096)[0], skip_special_tokens=True)
    return response

In [12]:
################################################################################
# Setting the same prompt which is used for GPT4 open AI
################################################################################

def get_user_question_for_open_ai_prompt(question):
    #return "Based on the following context:\n" + str(context) + "\n return a response for the question:\n" + question + "\n"
    return "\n Now return a response for this question:\n" + question + "\n"

def compose_prompt_open_ai_prompt(question,base_prompt=system_msg):
    return f"""[INST]<<SYS>>
            {base_prompt}
            <</SYS>>
            {get_user_question_for_open_ai_prompt(question)}
            [/INST]
            [END]"""
def post_process_open_ai_prompt_response(output):
    mql = output.split("[END]")[1].split("\n\nReasoning:\n\n")[0]
    reason = output.split("[END]")[1].split("\n\nReasoning:\n\n")[1]
    print("*"*50+"MQL"+"*"*50)
    print(mql)
    print("\n"+"*"*50+"Reason"+"*"*50)
    print(reason)
    #return mql,reason

In [16]:
%%time
question = "top 5 segments basis discount"
output = gereneate_output_llama2(compose_prompt_open_ai_prompt(question))
post_process_open_ai_prompt_response(output)

**************************************************MQL**************************************************


The response for the question "top 5 segments basis discount" is:

{
"MEASURE": {
"n-gram matched to MEASURE": [
    {
        "ENTITY": "Matched MEASURE",
        "MEASURE CONSTRAINT": [
            {
                "COMPARISON VALUE": "discount",
                "COMPARSION OPERATOR": ">"
            }
        ],
        "ADJECTIVE": [],
        "TONE": "positive"
    }
],
"DIMENSION": {
"n-gram matched to DIMENSION": [
    {
        "ENTITY": "Matched DIMENSION",
        "RANK": [
            {
                "RANK ADJECTIVE": "top",
                "RANK VALUE": "5"
            }
        ],
        "ADJECTIVE": [],
        "TONE": "positive"
    }
],
"FILTER": {
"n-gram matched to FILTER": [
    {
        "ENTITY": "Matched FILTER",
        "PARENT": "parent of the Matched FILTER",
        "EXCLUDE": ""
    }
],
"DERIVED MEASURE": {
"n-gram matched to DERIVED MEASURE": [
    

In [13]:
newprompt1 = """You are an assistant for a question answering system that analyzes user questions. Your task is to identify and extract various entities from the user's input and provide a structured response.

**Step 1: Entity Identification**
Your first task is to identify entities in the user's question. These entities include dimensions (e.g., "segments," "products"), measures (e.g., "sales," "discount"), adjectives (e.g., "top," "bottom"), ranks (e.g., "top 5," "bottom 3"), and other relevant elements.

**Step 2: Entity Relationships**
Once you identify these entities, determine any relationships between them. For instance, if the user asks for the "top 5 segments by sales," you should recognize "top" as the adjective, "5" as the rank value, "segments" as the dimension, and "sales" as the measure. Ensure that the relationships between these entities are correctly understood.

**Step 3: Generate a Structured Response**
Based on the entities identified and their relationships, construct a structured response. If adjectives, ranks, or other modifiers are present, include them in the response. The response should be structured into categories for dimensions, measures, and any other relevant details.

**Example:**
- User Input: "Show me the top 5 segments by sales."
- Expected Response: 
  {
      "DIMENSION": {
          "segments": [
              {
                  "ENTITY": "Segment",
                  "RANK": {
                      "RANK ADJECTIVE": "top",
                      "RANK VALUE": "5"
                  }
              }
          ]
      },
      "MEASURE": {
          "sales": [
              {
                  "ENTITY": "Sales"
              }
          ]
      }
  }

**Remember:**
- Be attentive to modifiers, ranks, and relationships between entities.
- Ensure the response is structured and contains relevant information based on the user's input.

Provide your response following this structure.
"""

In [15]:
print(output)

[INST]<<SYS>>
            You are an assistant for a question answering system that analyzes user questions. Your task is to identify and extract various entities from the user's input and provide a structured response.

**Step 1: Entity Identification**
Your first task is to identify entities in the user's question. These entities include dimensions (e.g., "segments," "products"), measures (e.g., "sales," "discount"), adjectives (e.g., "top," "bottom"), ranks (e.g., "top 5," "bottom 3"), and other relevant elements.

**Step 2: Entity Relationships**
Once you identify these entities, determine any relationships between them. For instance, if the user asks for the "top 5 segments by sales," you should recognize "top" as the adjective, "5" as the rank value, "segments" as the dimension, and "sales" as the measure. Ensure that the relationships between these entities are correctly understood.

**Step 3: Generate a Structured Response**
Based on the entities identified and their relationsh

In [14]:
%%time
question = "top 5 segments basis discount"
output = gereneate_output_llama2(compose_prompt_open_ai_prompt(question,base_prompt=newprompt1))
post_process_open_ai_prompt_response(output)

/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:2415: UserWarning: Specified kernel cache directory could not be created! This disables kernel caching. Specified directory is /home/mosaic-ai/.cache/torch/kernels. This warning will appear only once per process. (Triggered internally at ../aten/src/ATen/native/cuda/jit_utils.cpp:1442.)
  next_tokens.tile(eos_token_id_tensor.shape[0], 1).ne(eos_token_id_tensor.unsqueeze(1)).prod(dim=0)


IndexError: list index out of range

In [14]:
%%time
question = "top 5 segments basis discount"
output=gereneate_output_llama2(compose_prompt_open_ai_prompt(question))
post_process_open_ai_prompt_response(output)

**************************************************MQL**************************************************


The response for the question "top 5 segments basis discount" is as follows:

{
"MEASURE": {
"n-gram matched to MEASURE": [
{
"ENTITY": "Matched MEASURE",
"MEASURE CONSTRAINT": [
{
"COMPARISON VALUE": "top 5",
"COMPARISION OPERATOR": "="
}
],
"ADJECTIVE": [],
"TONE": ""
}
],
"DIMENSION": {
"n-gram matched to DIMENSION": [
{
"ENTITY": "Matched DIMENSION",
"RANK": [
{
"RANK ADJECTIVE": "top",
"RANK VALUE": "5"
}
],
"ADJECTIVE": [],
"TONE": ""
}
],
"FILTER": {
"n-gram matched to FILTER": [
{
"ENTITY": "Matched FILTER",
"PARENT": "Segment",
"EXCLUDE": ""
}
],
"DERIVED MEASURE": {
"n-gram matched to DERIVED MEASURE": [
{
"ENTITY": "Matched DERIVED MEASURE",
"RATIO FILTER": [
{}
],
"APPLIED MEASURE": [
{
"n-gram matched to MEASURE": [
{
"ENTITY": "Discount",
"RANK": [
{
"RANK ADJECTIVE": "top",
"RANK VALUE": "5"
}
]
}
],
"ADJECTIVE": [],
"TONE": ""
}
],
"DATE VARIABLE": {
"asked time ele

In [70]:
%%time
question = "discount rate of phone and binders"
output=gereneate_output_llama2(compose_prompt(question))
post_process(output)

**************************************************MQL**************************************************


The response for the question "discount rate of phone and binders" is:

{
"MEASURE": [
{
"ENTITY": "Discount",
"other names": ["discount", "discount rate", "discount value", "deduction"]
},
{
"ENTITY": "Purchase Vol",
"other names": ["purchase", "purchase value", "purchase model"]
},
{
"ENTITY": "Quantity",
"other names": ["quantity", "volume"]
},
{
"ENTITY": "Sales",
"other names": ["sales", "sale"]
}
],
"DIMENSION": [
{
"ENTITY": "Sub-Category",
"other names": ["sub-category", "sub category", "categories", "section"]
},
{
"ENTITY": "Segment",
"other names": ["segment", "segments", "units", "divisions"]
},
{
"ENTITY": "Parts",
"other names": ["parts", "part", "section", "divisions"]
},
{
"ENTITY": "Country",
"other names": ["country", "countries"]
}
],
"FILTER": [
{
"ENTITY": "Consumer",
"other names": ["consumers", "consumer"],
"parent": "Segment"
},
{
"ENTITY": "Phone",
"other n